In [ ]:
import numpy as np
import random
import matplotlib.pyplot as plt
import math

In [ ]:
random.seed(123)

# 1.誤差逆伝搬法の理解
ここのセクションでは，

$$
y = ax + b
$$

という関数について考えてみることにします．

## 1.1: 1変数の推定
まず，最も簡単なケースについて考えます．
ここで，
$
b = 2
$
であることが既知であり，
$
(x, y) = (2, 6) 
$
で式が成立することがわかっていたとします．

この時，
$$
6 = 2a + 2
$$
という関係式が成立し，$a=2$であることがわかります．まずは，この問題をあえて誤差逆伝搬法で解いてみましょう．

まずは，$ax + b$ をクラスで定義していきます．

In [ ]:
class Linear:
    def __init__(self, a_init: float, b_init: float):
        self.a: float = a_init
        self.b: float = b_init
        
    def forward(self, x: float) -> float:
        return #WRITE HERE

早速作ったクラスが正しく機能しているか見てみましょう．

In [ ]:
linear01 = Linear(a_init=2, b_init=2)
print("ax+b = 2x2+2 =", linear01.forward(2))
linear02 = Linear(a_init=4, b_init=1)
print("ax+b = 4x3+1 =", linear02.forward(3))

ここでは，Linearクラスとして実装を行いました．機械学習における各層は，基本的にクラスとして実装されます．これは，今回の場合，$a$が重み，$b$がバイアスと呼ばれるものであり，このLinear関数における内部変数として保存される必要があるためです．

では，続いて，どのように未知の値を推定していくか考えてみましょう．

今回の場合，$y=ax+b$という，きわめて簡単な式を用いていますので，簡単にできますが，あえて機械学習的に考えてみます．
「誤差逆伝搬法」というのは，あくまで「誤差」を定義し，これを逆向きにたどっていくことで，重みを修正していくわけです．
ですので，まずは，この「誤差」を定義します．


「誤差」は機械学習の世界においては「Loss」と言います．Loss関数というのを定め，学習を行います．
今回の場合は，$ax+b$が予測値，$y$が目標値のような関係です．そのため，今回のLoss関数は
$$
L(a,b)= y - (ax+b)
$$
と定義することとします．

今回は便宜上，重みは$a=2$で固定して，$b$を探索します．ですので，
$$
L(b) = y - Linear_{b}(x)
$$
となります．

ひとまず，bの初期値として，乱数で生成した値を使ってlossを計算してみましょう．

In [ ]:
random.seed(123)
b_init = random.random()
b_init

In [ ]:
linear_b = Linear("""WRITE HERE""")

In [ ]:
x, y = 2, 6
loss = #WRITE HERE
loss

これでは，まだ $Linear_{b}(x)$ のほうが小さいということになります．では，どうすればよいでしょうか？ $b$を大きくしますか？小さくしますか？どれだけ大きくすればちょうどよいですか？

In [ ]:
linear_b2 = Linear(a_init=2, b_init="""WRITE HERE""")
loss2 = #WRITE HERE
loss2

In [ ]:
linear_b2.b

これで，ほぼ正しく $b=2$を計算できましたね．ここでは，一気に答えに寄せに行きましたが，もう少し繰り返しを行って寄せていく方法もあります．

In [ ]:
linear_b3 = Linear(a_init=2, b_init=b_init)
learning_rate = 0.5
for i in range(100):
    loss = #WRITE HERE
    print(i, loss)
    linear_b3.b += loss * learning_rate

In [ ]:
linear_b3.b

こちらもほぼ誤差なく計算できましたね．
今回は厳密解のある計算でしたので，冗長に見えましたが，この，"loss"を計算して，それをパラメーター更新に用いるのが誤差逆伝搬法の考え方になります．

今回，誤差逆伝搬法において，パラメーターを更新していった方式を「最急降下法」といいます．これは，勾配の方向に最速で降下させていく方法です．

## 1.2 2変数の推定(OLS:最小二乗法)

先ほどは，1変数の推定でしたが，今度は2変数を推定することとします．
また，通常であれば，機械学習で使用するデータは1.1で確認したような完全に合致するようなケースは少なく，何らかの誤差を含みます．
そこで，ここでは誤差を含む複数のデータに基づく最小二乗法を考えます．

先ほどと同様の関数を用いますが，ここでは，以下のようにモデル化します．
$$
y=ax+b+\epsilon
$$
ここで，$\epsilon$は誤差項であり，正規分布を考えます．

ここでは，複数の$(x,y)$の組が与えられたときに，$a,b$を推測する問題になります．

まず，$(x,y)$の組を生成するにあたって，以下のように設定します．
$$
a = 2\\
b = 3\\
\epsilon \sim \mathscr{N}(0,1)
$$
とりあえず，$x$を$[0,5]$区間で設定して，データを生成します．

In [ ]:
datasets = []
random.seed(100)
for _ in range(10):
    _x = random.random()*5
    _y = 2 * _x + 3 + random.normalvariate(0, 1)
    datasets.append((_x, _y))
datasets

In [ ]:
X = np.array(list(map(lambda x: [x[0]], datasets)))
X = np.concatenate([np.ones_like(X), X], axis = -1)
Y = np.array(list(map(lambda x: [x[1]], datasets)))
X, Y

In [ ]:
plt.scatter(X[:, 1], Y)
plt.plot([0, 5], [3,13] )
plt.show()

いい感じにデータが生成できたのではないでしょうか？
上記のコードで
 - datasets: $(x,y)$のタプルのリスト
 - X: $[1, x]$の行列
 - Y: $y$の行列

を生成してあります．

ここでは，$a,b$の2変数なので，毎回書いてもそこまで問題はないですが，層の数が多い場合などの表記が大変ですので，
$$
h_\theta(x) = X\theta^T   =  b + aX
$$
と定義し，$\theta = [[a,b]]$とします．


では，この設定に基づいて，$a, b$を推定していきましょう．

まずは，最小化したいコスト関数 (=Loss関数) $J$を設定します．ここでは，最小二乗法にしたいので，MSEを用います．
MSEとは，mean square errorの略であり，
$$
J=L_{MSE} = \frac{1}{N}\sum_{i=1}^N(t_i - p_i)^2
$$
で定義されます．ここで，$t$はターゲットの意味で，$p$が予測値です．

では，MSE Lossを関数で定義してみましょう．

In [ ]:
def loss_MSE(inputs: np.ndarray, targets: np.ndarray) -> np.ndarray:
    if np.shape(inputs) != np.shape(targets):
        raise AssertionError
    if np.shape(inputs) != (len(inputs), 1):
        raise AssertionError
    return #WRITE HERE

In [ ]:
## 両方Trueになれば正解
print(loss_MSE(inputs=np.array([[2], [3]]), targets=np.array([[2], [3]])) == 0.0)
print(loss_MSE(inputs=np.array([[3], [2]]), targets=np.array([[2], [3]])) == 1.0)

では，次に，この問題での誤差逆伝搬を考えてみましょう．

結局のところ，loss関数を最小化するのが目的でした．では，最小化するにあたって，何をいじるのか，と考えたときに，いじるのは$\theta$です．

つまり，loss関数の$\theta$における勾配を計算することになります．
では，計算していきましょう．

ここまでの設定にしたがって定式化をすると，
$$
L = \frac{1}{N}\sum_{i=1}^N(y_i - h_\theta(x_i))^2
$$
となります．ただし，ここで，$N$個のデータを考えるのはいささか煩雑なので，とりあえず，$N=1$で考えてみましょう．
$$
L = (y - h_\theta(x))^2
$$
深層学習においては，すべての層の勾配をいっぺんに伝播させるのではなく，「連鎖」をさせて計算をします．つまり，勾配の逆関数をそれぞれの層に対して定式化しておき，それらを連鎖させていくことで，勾配の計算，パラメーターの更新を行います．

まず，MSE Lossの勾配を考えます．MSE Lossの勾配だけを考えると，上記の式の$L$を$h_\theta(x)$で微分したものになるはずです．
つまり，
$$
\frac{\partial L}{\partial h_\theta(x)} = -2(y - h_\theta(x))
$$
となります．

続いて，線形の層($h_\theta(x)$)について考えます．こちらは，若干煩雑です．

今回は，$x$が入力になるので，$x$に対する勾配を考えてもどうしようもありませんが，とりあえず考えてみると，
$$
\frac{\partial h_\theta(x)}{\partial x} = a
$$
となります．$\because h_\theta(x) = ax+b$

では，次にこの線形の層の$a, b$に対する勾配を考えてみましょう．
$$
\frac{\partial h_\theta(x)}{\partial a} = x
$$
$$
\frac{\partial h_\theta(x)}{\partial b} = 1
$$
となりますね．

では，なぜこれらの各層ごとの勾配を計算しておくだけでうまくいくのかを見ていきます．
これは，$L$に対する$a, b$の勾配を計算するだけですぐにわかります．

$$
\frac{\partial L}{\partial a} = \frac{\partial L}{\partial h_\theta(x)}\cdot\frac{\partial h_\theta(x)}{\partial a} = -2(y - h_\theta(x))x
$$
$$
\frac{\partial L}{\partial b} = \frac{\partial L}{\partial h_\theta(x)}\cdot\frac{\partial h_\theta(x)}{\partial b} = -2(y - h_\theta(x))
$$

しかし，深層学習においては，前述のとおりこの計算をいちいち行う必要はなく，今回の場合であれば，線形層がLoss関数から，勾配$-2(y - h_\theta(x))$を受け取りさえすれば充分なわけです．

今回は，$N=1$のケースで計算を行いましたが，実際には，$N$個のデータの勾配の平均を計算していきます．これをバッチ学習と言います．バッチ学習は，1つのデータセットに依存せずに複数の平均で勾配を計算しますので，学習が安定することなどのメリットがあるほか，GPUとの相性が良いため，標準的に使用されます．以下では，バッチ学習を前提として，実装を行います．

まず，今回は再急降下法の学習率$\alpha$は便宜上，グローバルに与えることとして，再急降下法のアルゴリズムおよびパラメーターの更新は各層の中の機能(updateという関数で，内部パラメーターを持つ層にのみ実装)に取り込む実装にしてしまいます．(実際には，多くの深層学習フレームワークは，勾配の伝搬だけを各層内のアルゴリズムに対して組み込み，パラメーター更新はoptimizerという形で，別の外部モジュールとして作成し，それが各層のパラメーターを更新していきます．)

In [ ]:
class MSELoss:
    def __init__(self):
        self.grad: np.ndarray = None
        pass
    
    def forward(self, inputs: np.ndarray, targets: np.ndarray) -> np.ndarray:
        self.grad = #WRITE HERE
        return loss_MSE(inputs=inputs, targets=targets)
    
    def backward(self) -> np.ndarray:
        return self.grad

In [ ]:
class Linear2:
    def __init__(self, alpha: float = 0.1):
        self.theta: np.ndarray = np.random.random([1,2])
        self.inputs: np.ndarray = None
        self.grad_theta: np.ndarray = None
        self.alpha: float = alpha
        
    def forward(self, inputs: np.ndarray) -> np.ndarray:
        ## inputs: N x 2 (X)
        ## outputs: N x 1
        if len(np.shape(inputs)) != 2:
            raise AssertionError("invalid dim")
        if np.shape(inputs)[1] != 2:
            raise AssertionError("invalid shape")
        self.inputs = inputs
        return #WRITE HERE
    
    def backward(self, grad: np.ndarray) -> np.ndarray:
        # grad: 1x1
        self.grad_theta: np.ndarray  = #WRITE HERE　行列演算だけで実装しましょう．a, bの勾配をいっぺんに計算できます
        return #WRITE HERE
    
    def update(self):
        self.theta -= self.alpha * self.grad_theta

In [ ]:
random.seed(123)
mse_layer = MSELoss()
linear_layer = Linear2()

In [ ]:
## 学習
loss_list = []
for i in range(100):
    ## forward
    pred = linear_layer.forward(inputs=X)
    loss = mse_layer.forward(inputs=pred, targets=Y)
    ## backward
    _grad = mse_layer.backward()
    _grad = linear_layer.backward(grad=_grad)
    ## update
    linear_layer.update()
    loss_list.append(loss)
    print(i, loss)

In [ ]:
plt.plot(loss_list)
plt.yscale('log')
plt.show()

きれいに収束しましたでしょうか？今回はノイズを含んでいますので，0にはなりません．では，$a, b$の値を最後に確認してみましょう．

In [ ]:
_a, _b = linear_layer.theta[0,1],linear_layer.theta[0,0] 
_a, _b

あれ，おかしいですね．想定していた$a, b$にはならなかったのではないでしょうか？
ただ，この値に基づいてプロットしてみましょう．

In [ ]:
plt.scatter(X[:, 1], Y)
plt.plot([0, 5], [_b, 5 * _a + _b] )
plt.show()

どうやら，近似自体は正しそうですので，正しい実装ができていそうに見えますね．

では，どうすれば，$a, b$を想定の値に近づけることができるでしょうか -> **課題1**

これで本章の内容はおしまいです．再急降下法や逆伝搬法，各層の関係はなんとなくわかりましたでしょうか？
実際に実装する際には，pytorchなどを使えば，もっと簡単にこれらの実装を行うことができます．
また，誤差逆伝搬については，autogradといった，自動で勾配計算するツールが入っていますので，基本的には理論さえわかっていれば，実装ができます．

また，補足的事項として，今回はMSEを使用して，学習を行いました．そのため，実質的に，この操作は最小二乗法の近似的計算を行っていることになります．実際にはこの問題は最小二乗法の計算を行えば，解析的に最適な係数を割り出すことが可能です．もし，興味があれば，計算してみてください．(計算自体は高校生の統計の授業で扱われるレベルの簡単な計算です．)今回Loss関数に使用したMSEが実質的な対数尤度と相似であることも確認できます．

# 2. クラス分類問題と活性化関数

今度は2クラス分類問題を考えます．
$$
 y = ax + b
$$
を境界線と考え，サンプルがたくさんある状況を考えます．

とりあえず，ここでは，$a=-1, b=3$のケースを考えます．

境界線付近は分離があいまいであるという仮定で，データを生成します．(ここでは詳細を割愛します．)

In [ ]:
X2 = []
T2 = []
random.seed(100)
for _ in range(1000):
    _x = random.random() * 3
    _y = random.random() * 3
    _diff = _y - (-1 * _x + 3)
    if _diff < -0.5:
        tag = 0
    elif _diff > 0.5:
        tag = 1
    else:
        if random.random() > _diff + 0.5:
            tag = 0
        else:
            tag = 1
    X2.append([_x, _y])
    T2.append([tag])
X2 = np.array(X2)
T2 = np.array(T2)

In [ ]:
plt.scatter(X2[(T2 == 0).reshape(-1)][:, 0], X2[(T2 == 0).reshape(-1)][:, 1], c="red")
plt.scatter(X2[(T2 == 1).reshape(-1)][:, 0], X2[(T2 == 1).reshape(-1)][:, 1], c="blue")
plt.plot([0,3],[3,0])
plt.show()

上記では，
 - X2: (x,y)のデータ．Nx2の形
 - T2: クラス分類のタグ (Nx1)

として，データを格納しています
では，これを使って，$x, y$が与えられたらタグを返す分類機を作ります．
もちろん，$a, b$は最後まで未知とします．

2クラス分類においてはsigmoidが良く使われます．sigmoidとは，
$$
f(x) = \frac{1}{1 + e^{-x}}
$$
と定義されます．では，関数で定義して，見てみましょう．

In [ ]:
def sigmoid(x):
    return #WRITE HERE

In [ ]:
plt.plot(np.linspace(-5,5,200), sigmoid(np.linspace(-5,5,200)))
plt.show()

sigmoid関数は，0.5を境として，2クラス分類に使われます．これは，簡単に言えばタグが1になる確率を表してるともいえるので，0.5を超えているかどうかは分類自体には重要ですが，その確度の情報として数値自体も学習時には重要です．

このsigmoidは「活性化関数」といわれるものの一種で，最終層に使われるだけでなく，途中の層でも使われることがあります．

クラス分類のロス関数としては，交差エントロピー(cross entropy)が良く使われ，他クラス分類にも使用できる形で
$$
-\sum_{i=1}^{C}t_i log{y_i}
$$
と定義されます．ここでは，$t_i$はタグのone-hotベクトル，$y_i$が$i$クラス目に対応する出力値です．
なお，他クラス分類の場合には，活性化関数はsoftmaxが使用されますが，計算精度向上のために，実際にはLogSoftmax + Negative Log-likelihood Loss (NLL Loss)の組み合わせで実装されることが多いです．

**[課題]** LogSoftmaxとNLL Lossについて調べてみよう．なぜこの組み合わせだとSoftmaxとCross entropy Lossの組み合わせに対して計算精度が良くなるのか考えてみよう．

今回は2クラス分類に使用しますが，sigmoidの出力は1つであるため，このcross entropyを特殊に加工したbinary cross entropy lossを使用します．
$$
y = sigmoid(h_\theta(X))
$$
で出てきた出力に対して，
$$
L = -\sum_{i=1}^{2}t_i log{y_i} = -(1-t)\cdot log{(1-y)} - t\cdot log {y}
$$
と定義されます．ここで，tは0/1のクラス分類値であるとします．

なお，ここで，$h_\theta(X)$という前回とほぼ同じノーテーションを使ってしまいましたが，これはこれまで同様に線形層を指し示しているものの，入力の大きさがことなり，今回は，$x$だけでなく$y$も入力となるため，1次元大きくなります．また，従来はバイアス項に対応する1をXに追加していましたが，今回はこれを追加せずに実装したいと思います．

つまり，
$$
h_\theta(X) = [1,X] \theta^T = c + ax + by
$$
とし，$\theta = [[c, a, b]]$となります．
では，これを実装していきます．

まずはLinear層です．ちょっとだけ実装が変わりますね．classの継承でスマートに対応してしまいましょう．

In [ ]:
class Linear3(Linear2):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.theta: np.ndarray = #WRITE HERE
        
    def forward(self, inputs: np.ndarray) -> np.ndarray:
        ## inputs: N x 2 (X)
        ## outputs: N x 1
        if len(np.shape(inputs)) != 2:
            raise AssertionError("invalid dim")
        if np.shape(inputs)[1] != 2:
            raise AssertionError("invalid shape")
        N = np.shape(inputs)[0]
        _inputs = np.concatenate([np.ones([N,1]), inputs], axis=1)
        self.inputs = _inputs
        return #WRITE HERE

続いて，sigmoidを作ってみましょう．逆伝搬を考えるために，微分を考えます．
$$
f(x) = \frac{1}{1 + e^{-x}}
$$
ですから，
$$
\frac{\partial f(x)}{\partial x} = ????(\mathrm{練習もかねて書いてみましょう})
$$
となります．ただし，これでは計算の無駄がありますので，一度計算した$f(x)$を効率的に使って書き換えてみると，
$$
\frac{\partial f(x)}{\partial x} = f(x)\cdot (1 - f(x)) 
$$
となり，勾配の計算は順方向の計算結果を用いれば極めて簡単に出すことができるとわかります．

では，これを使ってsigmoid層を実装してみましょう．

In [ ]:
class Sigmoid:
    def __init__(self):
        self.output = None
    
    def forward(self, inputs: np.ndarray) -> np.ndarray:
        self.output = sigmoid(inputs)
        return self.output
    
    def backward(self, grad: np.ndarray) -> np.ndarray:
        return #WRITE HERE

最後に，Binary Cross Entropy (BCE) Lossを実装します．
$$
L = -(1-t)\cdot log{(1-y)} - t\cdot log {y}
$$
ですので，これも微分して，
$$
\frac{\partial L}{\partial y} = ????(\mathrm{練習もかねて書いてみましょう})
$$
とします．ただし，理論上sigmoidのアウトプットが完全に$0$にならないので，$1-y$と$y$は$0$にならないはずで，0割りが発生しないはずですが，実際には計算の桁落ちで$0$になってしまい，nanを出してしまうので，分母が0にならないように非常に小さな$\epsilon$を足しておきます．ただし，今回の後述の実験では，そのような問題は実質起きないので，インスタンス作成時に$\epsilon=0$としています．(ここでの$\epsilon$は冒頭でデータの生成に使用した$\epsilon$とは違うものですが，一般に$\epsilon$と表記することが多いので，あえて使っています．)

In [ ]:
class BCELoss:
    def __init__(self, epsilon: float = 1e-10):
        self.grad: np.ndarray = None
        self.epsilon: float = epsilon
    
    def forward(self, inputs: np.ndarray, targets: np.ndarray) -> np.ndarray:
        self.grad = #WRITE HERE
        return #WRITE HERE
    
    def backward(self) -> np.ndarray:
        return self.grad

では，これらを用いて学習をさせていきたいと思います．

また，今回は性能確認のために，データセットを9:1に分けて，10%の分をテストに使ってみましょう．

In [ ]:
N_train = int(len(X2) * 0.9)
N_test = len(X2) - N_train
X2_train = X2[:N_train, :]
X2_test = X2[N_train:, :]
T2_train = T2[:N_train, :]
T2_test = T2[N_train:, :]

In [ ]:
random.seed(123)
bce_loss_layer = BCELoss(epsilon=0)
sigmoid_layer = Sigmoid()
linear_layer3 = Linear3(alpha=1)

In [ ]:
loss_list = []
for i in range(1000):
    ## forward
    _y = linear_layer3.forward(inputs=X2_train)
    _y = sigmoid_layer.forward(inputs=_y)
    loss = bce_loss_layer.forward(inputs=_y, targets=T2_train)
    ## backward
    _grad = bce_loss_layer.backward()
    _grad = sigmoid_layer.backward(grad=_grad)
    _grad = linear_layer3.backward(grad=_grad)
    ## update
    linear_layer3.update()
    loss_list.append(loss)
    print(i, loss)

In [ ]:
plt.plot(loss_list)
plt.yscale("log")
plt.show()

このcross entropyは非負実数であり，小さいほど良いものになります．正しく学習できていますでしょうか？

では，次にテストデータを使用して，性能を確認してみましょう．

In [ ]:
## forwardのみ
_y = linear_layer3.forward(inputs=X2_test)
pred = sigmoid_layer.forward(inputs=_y)
pred > 0.5

In [ ]:
TP = ((T2_test == 1) & (pred > 0.5)).sum()
FN = ((T2_test == 1) & (pred <= 0.5)).sum()
TN = ((T2_test == 0) & (pred <= 0.5)).sum()
FP = ((T2_test == 0) & (pred > 0.5)).sum()
prec = (TP) / (TP + FP)
rec = (TP) / (TP + FN)
print("Acc:", (TP + TN) / len(T2_test))
print("Precision:", prec)
print("Recall:", rec)
print("F1:", 2/(1/prec + 1/rec))

いかがでしたでしょうか？そこそこの精度が出ましたでしょうか？

# 3. 終わりに

第1章では回帰問題を，第2章では分類問題をフルスクラッチで実装しました．

ただ，今回扱ったのは単純な1つの線形層からなる判別機でしかありません．一般に，「深層学習」といった場合には，3層以上の層をもつものを言います．また，今回取り扱った活性化関数やロス関数もごくわずかなものですし，今回はパラメーター更新のアルゴリズムであるところのoptimizerに関しては再急降下法しか取り扱っていませんし，そのほかにも，Normalization, Grad clipなど，様々な学習の安定化方法もあります．きりがないほどたくさんありますが，それらを学び，適切なものを選択する能力が必要になります．

今回のように，勾配連鎖を実装することは極めてまれであり，基本的にはpytorchやtensorflowといったフレームワークがよきに計らってくれます．今回は，入門として，非常に簡単なものしか扱わないかわりに，フレームワークに依存することなく，numpyでフルスクラッチで実装してみました．しかし，今回取り上げた内容は深層学習の基礎であり，理論的背景を把握したうえでフレームワークを使用することは非常に有用です．特に勾配の連鎖に関しては，非常に重要な概念であり，今回の基礎が層が深くなっても適用可能です(実際にはフレームワークでは自分では実装しませんが．)．ぜひ，よい経験として，今回の内容をみなおし，今後の糧にしてもらえればと思います．

【課題】pytorchとtensorflowについて調べてみよう．今後機械学習を研究で使う可能性が高い場合には，どちらかを勉強してみよう．また，研究室のメンバーがどちらを使っているか聞いてみよう．